## Interference Testing

In [2]:
%matplotlib inline

import numpy as np
import RPi.GPIO as GPIO
from ring_buffer import RingBuffer
import matplotlib.pyplot as plt
import pickle

from ultrasonic_sensor import UltrasonicSensor

In [3]:
GPIO.cleanup()
GPIO.setup(4, GPIO.OUT)
GPIO.setup(17, GPIO.IN)
GPIO.setup(19, GPIO.IN)

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  """Entry point for launching an IPython kernel.


RuntimeError: Please set pin numbering mode using GPIO.setmode(GPIO.BOARD) or GPIO.setmode(GPIO.BCM)

# Collect Measurements

In [ ]:
import time
# intial parameters
n_iter = 1000


def measure():
    d1 = sensor1.distance()
    d2 = sensor2.distance()
    return d1, d2

sz = (n_iter,) # size of array
x = 35
zz = [measure() for _ in range(n_iter)]

In [ ]:
zz

In [ ]:
z1, z2 = list(zip(*zz))
plt.rcParams['figure.figsize'] = (15, 10)
plt.figure()
plt.plot(z1[50:],'r+',label='sensor 1')
plt.plot(z2[50:],'b+',label='sensor 2')

plt.xlabel('iteration')
plt.ylabel('distance')
plt.gca().set_ylim([0, 100])

# Run Filters

In [ ]:
start = 60
median_sample_size = 30

history1 = RingBuffer(capacity=median_sample_size, dtype=np.float)
history2 = RingBuffer(capacity=median_sample_size, dtype=np.float)
median1 = []
median2 = []
for z in z1:
    history1.append(z)
    med = np.median(history1)
    if med > start:
        med = start - (med - start)
    median1.append(med)
for z in z2:
    history2.append(z)
    med = np.median(history2)
    if med > start:
        med = start - (med - start)
    median2.append(med)

In [ ]:
plt.rcParams['figure.figsize'] = (15, 10)
plt.figure()
plt.plot(z1[50:],'r+',label='sensor 1')
# plt.plot(z2[50:],'b+',label='sensor 2')
plt.plot(median1[50:], 'r-', label='median 1')
# plt.plot(median2[50:], 'b-', label='median 2')
plt.legend()
plt.gca().set_ylim([0, 100])
plt.xlabel('iteration')
plt.ylabel('distance')

In [35]:
with open('two_sensors.pkl', 'wb') as f:
    pickle.dump(zz, f)

In [36]:
GPIO.cleanup()